In [ ]:
import numpy as np
import scipy.sparse.linalg as sla
import matplotlib.pyplot as plt
import refine_mesh
from time import time
from mesh_neu import read_neu
from LagrangeFE import *

Define the Poisson problem on a region $\Omega$:
$$ −\nabla \cdot \kappa(x, y) \, \nabla u(x, y) = f(x, x) $$
and $ u(x, y) = g(x, y) $ on $\partial \Omega$

In [ ]:
def f(x, y):
    # return 0
    if x**2 + y**2 > 0.25**2:
        return 0.0
    else:
        return 25.0

def kappa(x, y):
    # return 1
    if x**2 + y**2 > 0.25**2:
        return 0.1
    else:
        return 1.0

def g(x, y, tol=1e-12):
    if abs(y + 1) < tol:
        return 10.0 * (1 - x**2)
    else:
        return 0.0

In [ ]:
# Import a mesh
V, E = np.loadtxt('mesh.v'), np.loadtxt('mesh.e', dtype=int)
# V, E = refine_mesh.refine2dtri(V, E)

In [ ]:
# Linear LFE
print('------------  Linear  -----------')
print()
t0 = time()
LinearSolver = LinearLFE(V, E, f, kappa)
LinearSolver.integrator = quadD1S1
t1 = time()
AL, bL = LinearSolver.assembleMatrix()
t2 = time()
AL, bL = LinearSolver.implimentBoundary(AL, bL, g)
t3 = time()
uL = LinearSolver.solve(AL, bL)
t4 = time()
print( '     Timing:     ' )
print( 'Precompute Mesh:     ', round(1000 * (t1 - t0), 1) )
print( 'Assembly:            ', round(1000 * (t2 - t1), 1) )
print( 'Boundary Conditions: ', round(1000 * (t3 - t2), 1) )
print( 'Solving Matrix:      ', round(1000 * (t4 - t3), 1) )
print( 'Total:               ', round(1000 * (t4 - t0), 1) )
print('')
conda = sla.lsmr(AL, bL)[-2]
sparsity = 100 * (AL.nnz / (AL.shape[0] * AL.shape[1]))
print( '     Matrix Sparsity:     ' )
print( 'Nonzero Elements: ', AL.nnz )
print( 'Sparsity:         ', round(sparsity, 3) )
print( 'Condition Number: ', round(conda, 3))
# plot
fig = plt.figure(figsize=(6, 5))
plt.title('Linear Solution')
plt.triplot(V[:, 0], V[:, 1], E, 'k-', lw=0.25, ms=0)
plt.tripcolor(V[:, 0], V[:, 1], E, uL, shading='gouraud')
plt.colorbar()
plt.savefig('Linear.png', transparent=True, dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
# Quadratic LFE
print('----------  Quadratic  ----------')
print()
t0 = time()
# initializes the computational mesh and finds the boundary
QuadraticSolver = QuadraticLFE(V, E, f, kappa)
t1 = time()
# assembles the sparse matrix
AQ, bQ = QuadraticSolver.assembleMatrix()
t2 = time()
# impliments the boundry conditions on the sparse matrix
AQ, bQ = QuadraticSolver.implimentBoundary(AQ, bQ, g)
t3 = time()
# solves the linear problem
uQ = QuadraticSolver.solve(AQ, bQ)
t4 = time()
print( '     Timing:     ' )
print( 'Precompute Mesh:     ', round(1000 * (t1 - t0), 1) )
print( 'Assembly:            ', round(1000 * (t2 - t1), 1) )
print( 'Boundary Conditions: ', round(1000 * (t3 - t2), 1) )
print( 'Solving Matrix:      ', round(1000 * (t4 - t3), 1) )
print( 'Total:               ', round(1000 * (t4 - t0), 1) )
print('')
# the condition number of the matrix
conda = sla.lsmr(AQ, bQ)[-2]
# percent of non-zero entries
sparsity = 100 * (AQ.nnz / (AQ.shape[0] * AQ.shape[1]))
print( '     Matrix Sparsity:     ' )
print( 'Nonzero Elements: ', AQ.nnz )
print( 'Sparsity:         ', round(sparsity, 3) )
print( 'Condition Number: ', round(conda, 3))
# plot
fig = plt.figure(figsize=(6, 5))
plt.title('Quadratic Solution')
plt.triplot(V[:, 0], V[:, 1], E, 'k-', lw=0.25, ms=0)
plt.tripcolor(V[:, 0], V[:, 1], E, uQ[:len(V)], shading='gouraud')
plt.colorbar()
plt.savefig('Quadratic.png', transparent=True, dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
muL = np.mean(uL[E], 1)
muQ = np.mean(uQ[E], 1)
eA = np.array([ getArea(*V[ei]) for ei in E])
err = np.sqrt(np.sum(np.abs( eA * (muL - muQ)**2 )))
L2L = np.sqrt(np.sum(np.abs( eA * (muL)**2 )))
L2Q = np.sqrt(np.sum(np.abs( eA * (muQ)**2 )))
errP = 100 * (2 * err) / (L2L + L2Q)
print( '----------  Error  ----------' )
print( 'L2 Norm of Difference: ', round(err, 4) )
print( 'Percent Difference:    ', round(errP, 2) )
# plot difference
uQE = uQ[:len(V)]
uD = np.abs(uL - uQE)
fig = plt.figure(figsize=(6, 5))
plt.title('Difference Between Linear and Quadratic')
plt.triplot(V[:, 0], V[:, 1], E, 'k-', lw=0.25, ms=0)
plt.tripcolor(V[:, 0], V[:, 1], E, uD)
plt.colorbar()
plt.savefig('Difference.png', transparent=True, dpi=200, bbox_inches='tight')
plt.show()

In [ ]:
def exact(x, y, tol=1e-6):
    x, y, n, u = x + 1, y + 1, 1, 0
    while n < 200:
        sln_a = 160 * (1 - (-1)**n)
        sln_b = np.sin(n * np.pi * x / 2.)
        sln_c = np.sinh(n * np.pi * (2 - y) / 2.)
        sln_d = np.sinh(n * np.pi) * (n * np.pi)**3
        un = (sln_a * sln_b * sln_c) / sln_d
        u += un
        n += 1
        if (n > 10) and abs(un / u) < tol:
            return u
    return u


uE = 0 * V[:, 0]
for i, vi in enumerate(V):
    uE[i] = exact(vi[0], vi[1])
    
    
muL = np.mean(uL[E], 1)
muQ = np.mean(uQ[E], 1)
muE = np.mean(uE[E], 1)
eA = np.array([ getArea(*V[ei]) for ei in E])
errL = np.sqrt(np.sum(np.abs( eA * (muL - muE)**2 )))
errQ = np.sqrt(np.sum(np.abs( eA * (muQ - muE)**2 )))
print( 'Error Linear:    ', round(errL, 6) )
print( 'Error Quadratic: ', round(errQ, 6) )